In [1]:
from mdagent import MDAgent
import os 
import mdtraj as md

In [2]:
import sys
import os
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(parent_dir)
from robustness_prompts import get_prompt

prompt_3_descriptive = get_prompt("descriptive", 3)
prompt_3_natural = get_prompt("natural", 3)

prompt_3_descriptive, prompt_3_natural

('Complete all of the following tasks: 1. Download the PDB file 1LYZ. 2. Report the secondary structure assignments of the PDB structure. 3. Simulate for 1 ps at 300 K.',
 'Simulate 1LYZ for 1 ps at 300 K. Report the secondary structure assignments of the PDB structure.')

Required Tasks:
- Download
- SS Analysis
- Simulation

In [3]:
llm_model = "claude-3-5-sonnet-20240620"
tools = "all"

In [4]:
agent_1 = MDAgent(agent_type="Structured", model=llm_model, top_k_tools=tools)
agent_1.run(prompt_3_descriptive)

Certainly! I'll complete the tasks step by step using the provided tools.

Thought: First, we need to download the PDB file 1LYZ using the PDBFileDownloader tool.

Action:
```
{
    "action": "PDBFileDownloader",
    "action_input": "1LYZ"
}
```PDB file found with this ID: 1LYZ
Thought: Great, we have successfully downloaded the PDB file for 1LYZ. Now, let's report the secondary structure assignments of the PDB structure using the ComputeDSSP tool.

Action:
```
{
    "action": "ComputeDSSP",
    "action_input": {
        "traj_file": "1LYZ_172544",
        "top_file": "1LYZ_172544",
        "target_frames": "first"
    }
}
```

Thought: We have successfully obtained the secondary structure assignments for 1LYZ. Now, let's proceed with the final task of simulating the protein for 1 ps at 300 K using the SetUpandRunFunction tool.

Action:
```
{
    "action": "SetUpandRunFunction",
    "action_input": {
        "pdb_id": "1LYZ_172544",
        "forcefield_files": ["amber14-all.xml", "ambe

({'input': '\n    You are an expert molecular dynamics scientist, and\n    your task is to respond to the question or\n    solve the problem to the best of your ability using\n    the provided tools.\n\n    You can only respond with a single complete\n    \'Thought, Action, Action Input\' format\n    OR a single \'Final Answer\' format.\n\n    Complete format:\n    Thought: (reflect on your progress and decide what to do next)\n    Action:\n    ```\n    {\n        "action": (the action name, it should be the name of a tool),\n        "action_input": (the input string for the action)\n    }\n    \'\'\'\n\n    OR\n\n    Final Answer: (the final response to the original input\n    question, once all steps are complete)\n\n    You are required to use the tools provided,\n    using the most specific tool\n    available for each action.\n    Your final answer should contain all information\n    necessary to answer the question and its subquestions.\n    Before you finish, reflect on your pro

In [5]:
registry_1 = agent_1.path_registry
print(registry_1.list_path_names_and_descriptions().replace(",", "\n"))

Files found in registry: 1LYZ_172544: PDB file downloaded from RSCB
 PDBFile ID: 1LYZ_172544
 rec0_172547: dssp values for trajectory with id: 1LYZ_172544
 1LYZ_172600: Cleaned File:  Removed Heterogens
 and Water Removed.  Replaced Nonstandard Residues. Added Hydrogens at pH 7.0. Missing Atoms Added and replaces nonstandard residues. 
 top_sim0_172604: Initial positions for simulation sim0_172604
 sim0_172604: Basic Simulation of Protein 1LYZ_172600
 rec0_172605: Simulation trajectory for protein 1LYZ_172600 and simulation sim0_172604
 rec1_172605: Simulation state log for protein 1LYZ_172600 and simulation sim0_172604
 rec2_172605: Simulation pdb frames for protein 1LYZ_172600 and simulation sim0_172604


In [6]:
# make sure pdb was downloaded
assert os.path.exists(registry_1.get_mapped_path("1LYZ_172544"))

In [7]:
# make sure dssp was computed correctly
from mdagent.tools.base_tools import ComputeDSSP

dssp = ComputeDSSP(registry_1)
dssp._run(traj_file= "1LYZ_172544", target_frames="first")

"{'residues in helix': 50, 'residues in strand': 14, 'residues in coil': 65, 'residues not assigned, not a protein residue': 101}"

In [8]:
# make sure trajectory and topology exist
traj_path_1 = registry_1.get_mapped_path("rec0_172605")
top_path_1 = registry_1.get_mapped_path("top_sim0_172604")

assert os.path.exists(traj_path_1)
assert os.path.exists(top_path_1)

In [9]:
agent_2 = MDAgent(agent_type="Structured", model=llm_model, top_k_tools=tools)
agent_2.run(prompt_3_natural)

Thought: To simulate 1LYZ for 1 ps at 300 K and report the secondary structure assignments, I need to follow these steps:
1. Download the PDB file for 1LYZ
2. Set up and run a short simulation
3. Analyze the secondary structure of the PDB structure

Let's start by downloading the PDB file for 1LYZ.

Action:
```
{
    "action": "PDBFileDownloader",
    "action_input": "1LYZ"
}
```PDB file found with this ID: 1LYZ
Thought: Great, we have successfully downloaded the PDB file for 1LYZ. Now, let's set up and run a short simulation for 1 ps at 300 K using the SetUpandRunFunction tool.

Action:
```
{
    "action": "SetUpandRunFunction",
    "action_input": {
        "pdb_id": "1LYZ_172917",
        "forcefield_files": ["amber14-all.xml", "amber14/tip3pfb.xml"],
        "system_params": {
            "nonbondedMethod": "NoCutoff",
            "constraints": "HBonds",
            "rigidWater": True
        },
        "integrator_params": {
            "integrator_type": "LangevinMiddle",
      

ValidationError: 1 validation error for SetUpandRunFunctionInput
forcefield_files
  Field required [type=missing, input_value={'pdb_id': "pdb_id=1LYZ_1...ergy', 'temperature']}"}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.8/v/missing

In [10]:
registry_2 = agent_2.path_registry
print(registry_2.list_path_names_and_descriptions().replace(",", "\n"))

Files found in registry: 1LYZ_172917: PDB file downloaded from RSCB
 PDBFile ID: 1LYZ_172917


In [12]:
# make sure pdb was downloaded
assert os.path.exists(registry_2.get_mapped_path("1LYZ_172917"))

In [14]:
# # make sure dssp was computed correctly
# from mdagent.tools.base_tools import ComputeDSSP

# dssp = ComputeDSSP(registry_2)
# dssp._run(traj_file= "1LYZ_172917", target_frames="first")

In [ ]:
## make sure trajectory and topology exist
# traj_path_2 = registry_2.get_mapped_path("<traj_path>")
# top_path_2 = registry_2.get_mapped_path("<top_path>")

# assert os.path.exists(traj_path_2)
# assert os.path.exists(top_path_2)